In [1]:
# libs
import os
import pandas as pd
import requests
from dotenv import load_dotenv #pip3 install python-dotenv

# load secrets
load_dotenv()

True

In [ ]:
iex_key = os.getenv("IEX_API_KEY")
url = 'https://cloud.iexapis.com/stable/ref-data/symbols?token=' + iex_key

# Make the API request
response = requests.get(url)
data = response.json()

# Convert the data into a pandas DataFrame
symbols_df = pd.DataFrame(data)

# Assuming the response includes symbols and basic company info,
# you can filter or process this DataFrame further as needed.
print(symbols_df.head(50))  # Display the first few rows to verify


In [10]:

# DEV - output to csv
symbols_df.to_csv('symbols.csv', index=False)  # Save the data to a CSV file

In [7]:
# DEV - get total symbols
len(symbols_df.index)

10951

In [4]:
# testing the api connection with historical data
def get_historical_data(symbol, lookback_days=5):
    iex_key = os.getenv("IEX_API_KEY") 
    historical_url = f'https://cloud.iexapis.com/v1/stock/{symbol}/chart/{lookback_days}d?token={iex_key}'

    historical_data = requests.get(historical_url).json()
    df = pd.DataFrame(historical_data)
    return df[['date', 'high', 'low', 'open','close']]

# Fetch historical data
ticker = 'PLTR' # Palantir Technologies Inc
#### sudz4 -> think about getting a list, can do list of SP500 or sector, pass the list to the function and get the data
historical_df = get_historical_data(ticker)

# Display the historical data
# print(historical_df)

# Convert the 'date' column to datetime objects
historical_df['date'] = pd.to_datetime(historical_df['date'])

# Sort the DataFrame by the 'date' column
historical_df = historical_df.sort_values(by='date')

In [5]:
historical_df 

,date,high,low,open,close
0,2024-02-08,25.060,23.32,23.96,24.51
1,2024-02-09,25.110,23.57,25.04,24.38
2,2024-02-12,25.525,23.82,23.91,25.05
